# PowerShell User Group innsalzach 08.07.2021

## PowerShell DSC with Azure Automation DSC

## What is DSC?

DSC (Desired State Configuration) is a platform where you can manage and develop your infrastructure with Configuration as Code. You write a DSC Configuration in a declarative way to describe the target state of your Infrastructure.

### Declarative vs. Imperative Code

I used declarative above but have not told you about what that means.

**Imperative:**

PowerShell executes what we develop and executes It. We as developers have to for example care about the error handling and the repetetive execution. An example of imperative code would be:

In [ ]:
#Check and install ASP.NET 4.5 feature
If (-not (Get-WindowsFeature "Web-Asp-Net45").Installed) {
    try {
        Add-WindowsFeature Web-Asp-Net45
    }
    catch {
        Write-Error $_
    }
}

**Declarative:**

We as developers define what needs to be done and not how it needs to be done. We do not care how to get to the desired state. The underlying platform (DSC) has to know that. An example of declarative code would be:

In [ ]:
Configuration WebSiteConfig
{
    Node MyWebServer
    {
        WindowsFeature IIS
        {
            Ensure = "Present"
            Name = "Web-Asp-Net45"
        }      
    }
}

### Important Terms in DSC

**Configurations:**

The declarative PowerShell scripts which define the desired state for the infrastructure. The configurations calls the DSC resources to bring the system in the desired state ("make it so").

They get "compiled" into a MOF (Managed Object Format) - File. The MOF - File is executed by the LCM.

**Resource:**

They contain the code that put and keep the target of a configuration in the specified state.

Resouces are PowerShell modules with a specific file structure.

Every resource has the following functions / methods:

- Get
- Set
- Test

**LCM (Local Configuration Manager):**

This is the core of PowerShell DSC. It is installed on every Windows System that has PowerShell 4.0 and above installed. It executes the MOF - Files and brings the infrastructure in its desired state. If the infrastructure is not in the desired state the LCM will trigger the last know configuration and put the system back in the desired state (Configuration drift).

  

The state of the LCM can be checked with

In [ ]:
Get-DscLocalConfigurationManager

If you want to configure the LCM you have to write a special mof file (meta.mof).

After that you execute

In [ ]:
Set-DscLocalConfigurationManager -Path <Path to meta.mof (not with file name)>

If you want to read further [Configure LCM](https://nedimmehic.org/2019/05/06/desired-state-configuration-dsc-local-configuration-manager-lcm/).

**Configuration Data**

In DSC you normally seperate the configuration and the configuration data. The configuration data is a .ps1 - File and the configuration data is a .psd1 file (PowerShell Data File)

  

What that means is that you can reuse your DSC Configuration across multiple environments (DEV, TEST, PROD) and only replace the configuration Data (configuration Information). A sample configuration Data file:

In [ ]:
$MyData =
@{
    AllNodes =
    @(
        @{
            NodeName           = "*"
            LogPath            = "C:\Logs"
        },

        @{
            NodeName = "VM-1"
            SiteContents = "C:\Site1"
            SiteName = "Website1"
        },


        @{
            NodeName = "VM-2"
            SiteContents = "C:\Site2"
            SiteName = "Website2"
        }
    );

    NonNodeData =
    @{
        ConfigFileContents = (Get-Content C:\Template\Config.xml)
     }
}

The configuration File referencing the configuration data

In [ ]:
configuration WebsiteConfig
{
    Import-DscResource -ModuleName xWebAdministration

    node $AllNodes.NodeName
    {
        xWebsite Site
        {
            Name         = $Node.SiteName
            PhysicalPath = $Node.SiteContents
            Ensure       = "Present"
        }

        File ConfigFile
        {
            DestinationPath = $Node.SiteContents + "\\config.xml"
            Contents = $ConfigurationData.NonNodeData.ConfigFileContents
        }
    }
}

Configuration data can get complex very fast. If you want to manage your configuration data right please use Gaels module [Datum.](https://github.com/gaelcolas/Datum)

  

Also please have a look at the DSC Workshop repository ([DSCWorkshop](https://github.com/dsccommunity/DscWorkshop/)). This project uses Datum to build the MOF-Files.

**Push and Pull Configuration**